# ml_lstm: End-to-end CLI workflow (ingestion òÆÒ paper trading)This notebook collects the major project CLIs needed to run the full workflow from data ingestion to paper trading.High-level steps:1. Run the daily data pipeline (ingestion, cleaning, gap handling, feature engineering).2. Train the LSTM model.3. Evaluate the latest best model.4. Generate per-bar predictions CSV.5. Run a backtest using that CSV.6. Plot backtest diagnostics.7. Run simulated paper trading using the same CSV.All `!` shell commands assume this notebook is in `notebooks/` and the project root is the parent directory.

In [9]:
import sys, subprocess; from pathlib import Path; import importlib; project_root = Path().resolve().parent; import src.config as cfg; cfg = importlib.reload(cfg); from src.config import (get_predictions_csv_path, get_trades_csv_path, get_equity_csv_path, get_hourly_data_csv_path); FREQUENCY = cfg.FREQUENCY; TSTEPS = cfg.TSTEPS; PREDICTIONS_CSV = get_predictions_csv_path('nvda', FREQUENCY); TRADES_CSV = get_trades_csv_path('nvda', FREQUENCY); EQUITY_CSV = get_equity_csv_path('nvda', FREQUENCY); PRICE_CSV = get_hourly_data_csv_path(FREQUENCY); print('Using FREQUENCY =', FREQUENCY, 'TSTEPS =', TSTEPS); print('PREDICTIONS_CSV =', PREDICTIONS_CSV); print('TRADES_CSV      =', TRADES_CSV); print('EQUITY_CSV      =', EQUITY_CSV); print('PRICE_CSV       =', PRICE_CSV); def run_module(mod, *args): cmd = [sys.executable, '-m', mod, *map(str, args)]; print('Running:', ' '.join(cmd), 'in', project_root); result = subprocess.run(cmd, cwd=project_root, check=False); if result.returncode != 0: print(f'Command failed with exit code {result.returncode}'); return result.returncode

SyntaxError: invalid syntax (3007044118.py, line 1)

In [10]:
import sys!{sys.executable} -m pip install tqdm

SyntaxError: invalid syntax (3620515485.py, line 1)

In [11]:
# Legacy path setup cell kept for backwards compatibility. The main configuration and path setup now happens in the first code cell. pass

## 1. Daily data pipeline (ingestion òÆÒ processed hourly features)Runs `src.daily_data_agent` which orchestrates ingestion, cleaning, gap handling, curated minute snapshot, and resampling + feature engineering.

In [ ]:
# Run the daily data pipeline (ingestion -> processed features). For a dry run that skips IB/TWS ingestion, add '--skip-ingestion' to the arguments below. run_module('src.daily_data_agent')

## 2. Train LSTM model

In [ ]:
# Train an LSTM model for the chosen FREQUENCY and TSTEPS using src.train. run_module('src.train', '--frequency', FREQUENCY, '--tsteps', TSTEPS)

## 3. Evaluate latest best modelEvaluate the latest best model (based on `best_hyperparameters.json`) using `src.evaluate_model`.

In [ ]:
# Evaluate the latest best model using src.evaluate_model. run_module('src.evaluate_model')

## 4. Generate per-bar predictions CSVUse `scripts.generate_predictions_csv` to create a per-bar predictions file for NVDA at the chosen `FREQUENCY`.Output columns: `Time`, `predicted_price`.

In [ ]:
# Generate per-bar predictions CSV for NVDA at the chosen FREQUENCY. run_module('scripts.generate_predictions_csv', '--frequency', FREQUENCY, '--output', PREDICTIONS_CSV)

## 5. Run backtest using predictions CSVRun `src.backtest` over processed OHLC data, using the predictions CSV as the prediction source.- `--prediction-mode csv` tells the backtest to read `predicted_price` from the CSV.- Exports trades and equity curve CSVs for diagnostics.

In [ ]:
# Run a backtest over processed OHLC data using the predictions CSV. run_module('src.backtest', '--frequency', FREQUENCY, '--prediction-mode', 'csv', '--predictions-csv', PREDICTIONS_CSV, '--export-trades-csv', TRADES_CSV, '--export-equity-csv', EQUITY_CSV)

## 6. Plot backtest diagnosticsUse `scripts.plot_backtest_diagnostics` to visualize equity curve and trade entry density.

In [ ]:
# Legacy CLI-based diagnostics; prefer the next cell which calls the plotting helper directly. pass

In [ ]:
from pathlib import Path; from scripts.plot_backtest_diagnostics import plot_equity_and_trade_density; from src.config import FREQUENCY, get_equity_csv_path, get_trades_csv_path, get_hourly_data_csv_path; project_root = Path().resolve().parent; equity_path = project_root / get_equity_csv_path('nvda', FREQUENCY); trades_path = project_root / get_trades_csv_path('nvda', FREQUENCY); price_path = project_root / get_hourly_data_csv_path(FREQUENCY); print('Equity CSV:', equity_path, 'exists:', equity_path.exists()); print('Trades CSV:', trades_path, 'exists:', trades_path.exists()); print('Price CSV:', price_path, 'exists:', price_path.exists()); plot_equity_and_trade_density(equity_path=str(equity_path), trades_path=str(trades_path), price_csv_path=str(price_path))

## 7. Paper trading over historical dataUse `src.paper_trade` to run a simulated paper-trading session over historical NVDA data, reusing the predictions CSV.

In [ ]:
!cd .. && python -m src.paper_trade --frequency {FREQUENCY} --predictions-csv {PREDICTIONS_CSV}

## 8. (Optional) Hyperparameter tuning and experiment gridNot required for the minimal ingestion òÆÒ paper-trading loop, but useful for experimentation.

In [ ]:
# Optional: run hyperparameter tuning (can be long-running)# !cd .. && python -m src.tune_hyperparameters# Optional: run a grid of experiments (very long-running)# !cd .. && python -m src.experiment_runner